# Demo notebook to evaluate pre-trained model from ML4Fires

First import relevant libraries to run the inference

In [ ]:
from tqdm import tqdm
import torch
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from Fires._utilities.utils_mlflow import load_model_from_mlflow
from Fires._utilities.utils_inference import load_input_data, create_data_loader, process_and_plot_data, get_prov_image

Select the model by specfying the run id from the MLFlow and download model, scaler and provenance document

In [ ]:
run_id=input()
registered_model = load_model_from_mlflow(run_id, provenance=True)
registered_model 

Let's check ML model provenance

In [ ]:
prov_image = get_prov_image(run_id)
from IPython.display import Image
Image(filename=prov_image) 

Let's get the data for the model evaluation

In [ ]:
# define path to complete dataset
data_path = "../ML4Fires_data/data_100km.zarr"
input_data = load_input_data(data_path, '2019', '2020')
input_data

In [ ]:
torch_data_loader = create_data_loader(data_path, run_id)

Let's run the prediction with the trained model downloaded from MLFlow

In [ ]:
preds = []
with torch.no_grad():
	for data, _ in tqdm(torch_data_loader):
		prediction = registered_model(data.to('cuda:0'))
		prediction_cpu = prediction.cpu().numpy()
		preds.append(prediction_cpu)
preds_array = np.vstack(preds)

Let's plot the actual burned area data from the test set

In [ ]:
process_and_plot_data(
	data=input_data.fcci_ba,
	label='FCCI Burned Areas',
	lats=input_data.latitude.values,
	lons=input_data.longitude.values,
	model_name="Observations"
)

Let's plot the predicted burned areas from the ML model

In [ ]:
process_and_plot_data(
	data=preds_array,
	label='Predicted Burned Areas',
	lats=input_data.latitude.values,
	lons=input_data.longitude.values,
	model_name="Unet ++"
)